<a href="https://colab.research.google.com/github/Pathway2008/sub_pred/blob/main/sub_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
info = pd.read_csv('/content/drive/MyDrive/predict_subcribe/data_info.csv')
sub = pd.read_csv('/content/drive/MyDrive/predict_subcribe/sample_submission.csv')
train = pd.read_csv('/content/drive/MyDrive/predict_subcribe/train.csv')
test = pd.read_csv('/content/drive/MyDrive/predict_subcribe/test.csv')

In [23]:
train = train.drop(['user_id'],axis=1)

In [24]:
train = train.drop(['target'],axis=1)

In [25]:
test = test.drop(['user_id'],axis=1)

In [26]:
from sklearn.preprocessing import LabelEncoder
labels = [col for col in train.columns if train[col].dtype == 'object' and col not in ['user_id']]
label_encoders = {}
for col in labels:
    label_encoders[col] = LabelEncoder()
    train[col] = label_encoders[col].fit_transform(train[col])
    test[col] = label_encoders[col].transform(test[col])

In [27]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

train["is_train"] = 0
test["is_train"]  = 1

In [28]:
df = pd.concat([train, test], axis = 0)
df = df.sample(frac = 1, random_state = 123)

In [29]:
X = df.drop(['is_train'], axis = 1)
y = df['is_train']

In [30]:
import pandas as pd
from xgboost import XGBClassifier

In [31]:
xgb_model = XGBClassifier(random_state=42, max_depth=2, scale_pos_weight=sum(y==0)/sum(y==1))
xgb_model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [32]:
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(y, xgb_model.predict_proba(X)[:,1])
print(auc_score)

0.646921505


In [33]:
feature_imp_xgb = pd.DataFrame({
    'Feature': list(X.columns),
    'XGB_Score': xgb_model.feature_importances_
})

In [34]:
feature_imp_xgb = feature_imp_xgb.sort_values(by='XGB_Score', ascending=False)
feature_imp_xgb

,Feature,XGB_Score
9,preferred_difficulty_level,0.473978
10,subscription_type,0.259503
1,recent_login_time,0.033135
5,total_completed_courses,0.032388
7,abandoned_learning_sessions,0.031885
0,subscription_duration,0.029616
3,average_time_per_learning_session,0.028920
2,average_login_time,0.022149
12,payment_pattern,0.020712
4,monthly_active_learning_days,0.020666


In [35]:
train.drop(['preferred_difficulty_level','subscription_type'],axis=1,inplace=True)
test.drop(['preferred_difficulty_level','subscription_type'],axis=1,inplace=True)

In [36]:
train.drop(['community_engagement_level'],axis=1,inplace=True)
test.drop(['community_engagement_level'],axis=1,inplace=True)

In [37]:
df = pd.concat([train, test], axis = 0)
df = df.sample(frac = 1, random_state = 123)

X = df.drop(['is_train'], axis = 1)
y = df['is_train']

In [38]:
import pandas as pd
from xgboost import XGBClassifier

xgb_model = XGBClassifier(random_state=42, max_depth=2, scale_pos_weight=sum(y==0)/sum(y==1))
xgb_model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [39]:
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(y, xgb_model.predict_proba(X)[:,1])
print(auc_score)

0.59958738


In [40]:
feature_imp_xgb = pd.DataFrame({
    'Feature': list(X.columns),
    'XGB_Score': xgb_model.feature_importances_
})

feature_imp_xgb = feature_imp_xgb.sort_values(by='XGB_Score', ascending=False)
feature_imp_xgb

,Feature,XGB_Score
7,abandoned_learning_sessions,0.142546
4,monthly_active_learning_days,0.131275
1,recent_login_time,0.114044
5,total_completed_courses,0.109809
0,subscription_duration,0.105182
3,average_time_per_learning_session,0.096226
2,average_login_time,0.088168
6,recent_learning_achievement,0.078415
8,customer_inquiry_history,0.070863
9,payment_pattern,0.063471


In [41]:
X_new = X.copy()
X_new['proba'] = xgb_model.predict_proba(X)[:,1]
X_new['target'] = y

In [42]:
X_new = X_new[X_new['target']==0]

In [43]:
nrows = X_new.shape[0]
adversarial_validation_data = X_new.sort_values(by='proba',ascending=False)[:int(nrows*.3)]
adversarial_training_data = X_new.sort_values(by='proba',ascending=False)[int(nrows*.3):]

In [44]:
adversarial_training_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7000 entries, 5800 to 5803
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              7000 non-null   int64  
 1   recent_login_time                  7000 non-null   int64  
 2   average_login_time                 7000 non-null   float64
 3   average_time_per_learning_session  7000 non-null   float64
 4   monthly_active_learning_days       7000 non-null   int64  
 5   total_completed_courses            7000 non-null   int64  
 6   recent_learning_achievement        7000 non-null   float64
 7   abandoned_learning_sessions        7000 non-null   int64  
 8   customer_inquiry_history           7000 non-null   int64  
 9   payment_pattern                    7000 non-null   int64  
 10  proba                              7000 non-null   float32
 11  target                             7000 non-null   int64  

In [45]:
current_index_order = adversarial_training_data.index.tolist()
valid_index = adversarial_validation_data.index.tolist()

In [49]:
x_train = train.loc[current_index_order].reset_index(drop=True)
x_valid = train.loc[valid_index].reset_index(drop=True)

In [51]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   user_id                            7000 non-null   object 
 1   subscription_duration              7000 non-null   int64  
 2   recent_login_time                  7000 non-null   int64  
 3   average_login_time                 7000 non-null   float64
 4   average_time_per_learning_session  7000 non-null   float64
 5   monthly_active_learning_days       7000 non-null   int64  
 6   total_completed_courses            7000 non-null   int64  
 7   recent_learning_achievement        7000 non-null   float64
 8   abandoned_learning_sessions        7000 non-null   int64  
 9   community_engagement_level         7000 non-null   int64  
 10  preferred_difficulty_level         7000 non-null   object 
 11  subscription_type                  7000 non-null   objec

In [1]:
pip install autogluon

In [41]:
x_train.to_csv('xgb_train.csv',index=False)
x_valid.to_csv('xgb_valid.csv',index=False)

In [52]:
x_train = pd.read_csv('/content/xgb_train.csv')
x_valid = pd.read_csv('/content/xgb_valid.csv')

In [53]:
x_train['harder'] = 0
x_train.loc[(x_train['average_time_per_learning_session'] > 167), 'harder'] = 1

x_valid['harder'] = 0
x_valid.loc[(x_valid['average_time_per_learning_session'] > 167), 'harder'] = 1

test['harder'] = 0
test.loc[(test['average_time_per_learning_session'] > 167), 'harder'] = 1

In [54]:
x_train['degree'] = x_train['monthly_active_learning_days'] * x_train['average_time_per_learning_session']
x_train['concentrate'] = x_train['average_time_per_learning_session'] - x_train['average_login_time']

x_valid['degree'] = x_valid['monthly_active_learning_days'] * x_valid['average_time_per_learning_session']
x_valid['concentrate'] = x_valid['average_time_per_learning_session'] - x_valid['average_login_time']

test['degree'] = test['monthly_active_learning_days'] * test['average_time_per_learning_session']
test['concentrate'] = test['average_time_per_learning_session'] - test['average_login_time']

In [55]:
x_train.drop(['user_id'],axis=1,inplace=True)
x_valid.drop(['user_id'],axis=1,inplace=True)

In [56]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              7000 non-null   int64  
 1   recent_login_time                  7000 non-null   int64  
 2   average_login_time                 7000 non-null   float64
 3   average_time_per_learning_session  7000 non-null   float64
 4   monthly_active_learning_days       7000 non-null   int64  
 5   total_completed_courses            7000 non-null   int64  
 6   recent_learning_achievement        7000 non-null   float64
 7   abandoned_learning_sessions        7000 non-null   int64  
 8   community_engagement_level         7000 non-null   int64  
 9   preferred_difficulty_level         7000 non-null   object 
 10  subscription_type                  7000 non-null   object 
 11  customer_inquiry_history           7000 non-null   int64

In [57]:
from autogluon.tabular import TabularDataset, TabularPredictor
train = TabularDataset(x_train)
valid = TabularDataset(x_valid)
test = TabularDataset(test)

In [58]:
fit_parameters = {
    'presets': 'best_quality',
    'num_bag_folds': 7,
    'num_bag_sets': 5,
    'num_stack_levels': 2
}

In [ ]:
predictor = TabularPredictor(
    label='target',
    eval_metric='f1_macro',
    sample_weight='balance_weight'
)

predictor.fit(
    train_data=train,
    tuning_data=valid,
    auto_stack=True,
    fit_weighted_ensemble=True,
    use_bag_holdout=True,
    presets=fit_parameters,
    num_gpus=1,
    hyperparameter_tune_kwargs='auto'
)